# 1.
   - Has the network latency gone up since we switched internet service providers?
  

In [ ]:
# null hypothesis 
"""
There is no statistically significant difference in network latency time between service providers. 
"""
# alternate hypothesis
"""
There is a statistically significant difference in network latency time between service providers.
"""
# true positive (REJECT the null, and the null is FALSE)
"""
When latency was tested a p-value lower than our significance level (0.05) was identified.
(i.e. evidence enough to support the alternative) so and must reject the null hypothesis. 
"""
# true negative (ACCEPT the null, and the null is TRUE)
"""
When latency was tested a p-value greater than our significance level (0.05) was identified.
(i.e. evidence enough to support the null) so we fail to reject the null hypothesis.
"""
# type I error (REJECT the null hypothesis, but, in reality, the null hypothesis is TRUE)
"""
When latency was tested a p-value lower than our significance level (0.05) was identified.
(i.e. evidence enough to support the alternative) but we failed reject the null hypothesis. 
"""
# type II error (ACCEPT the null hypothesis when it is actually FALSE.)
"""
When latency was tested a p-value greater than our significance level (0.05) was identified.
(i.e. evidence enough to support the null) but we rejected the null hypothesis.
"""

# 2. 
   - Is the website redesign any good?
   

In [ ]:
# null hypothesis
"""
There is no statistically significant quantifiable difference between the old and new design.
"""
# alternate hypothesis
"""
There is a statistically significant quantifiable difference between the old and new design.
"""
# true positive
"""
When quantifiable differences were tested a p-value lower than our significance level (0.05) was identified.
(i.e. evidence enough to support the alternative) so and must reject the null hypothesis.
"""
# true negative
"""
When quantifiable differences were tested a p-value greater than our significance level (0.05) was identified.
(i.e. evidence enough to support the null) so we fail to reject the null hypothesis.
"""
# type I error
"""
When quantifiable differences were tested a p-value lower than our significance level (0.05) was identified.
(i.e. evidence enough to support the alternative) but we failed reject the null hypothesis.
"""
# type II error
"""
When quantifiable differences were tested a p-value greater than our significance level (0.05) was identified.
(i.e. evidence enough to support the null) but we rejected the null hypothesis.
"""

# 3.   
   - Is our television ad driving more sales?

In [ ]:
# null hypothesis
"""
The ad has not had a statistically significant impact on sales
"""
# alternate hypothesis
"""
The ad has provided a statistically significant impact on sales
"""
# true positive
"""
When sales were examined a p-value lower than our significance level (0.05) was identified.
(i.e. evidence enough to support the alternative) so and must reject the null hypothesis.
"""
# true negative
"""
When sales were examined a p-value greater than our significance level (0.05) was identified.
(i.e. evidence enough to support the null) so we fail to reject the null hypothesis.
"""
# type I error
"""
When sales were examined a p-value lower than our significance level (0.05) was identified.
(i.e. evidence enough to support the alternative) but we failed reject the null hypothesis.
"""
# type II error
"""
When sales were examined a p-value greater than our significance level (0.05) was identified.
(i.e. evidence enough to support the null) but we rejected the null hypothesis.
"""